In [1]:
from azureml.core import Workspace, Run
from azureml.core.model import Model
from azureml.core.conda_dependencies import CondaDependencies 
from azureml.core.image import ContainerImage
from azureml.core.webservice import Webservice, AciWebservice

Connect to the Azure Machine Learning Services Workspace using information in config.json. If not yet authenticated, you will be redirected to the authentication page.

In [2]:
ws = Workspace.from_config()

Define and deploy prediction runtime container image.

In [3]:
myenv = CondaDependencies()
myenv.add_conda_package("keras")
myenv.add_conda_package("tensorflow=1.13.1")

with open("conda-config.yml","w") as f:
    f.write(myenv.serialize_to_string())

image_config = ContainerImage.image_configuration(execution_script="predict.py", 
                                                  runtime="python",
                                                  conda_file="conda-config.yml")

In [4]:
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               description='AI Lunch&Learn Demo')

In [5]:
model=Model(ws, 'fashion-mnist-model')

In [6]:
%%time
service = Webservice.deploy_from_model(workspace=ws,
                                       name='ai-demo-svc',
                                       deployment_config=aciconfig,
                                       models=[model],
                                       image_config=image_config)

Creating image
Running......................................................
Succeeded
Image creation operation finished for image ai-demo-svc:9, operation "Succeeded"
Creating service
Wall time: 4min 57s


Wait for the service to start.

In [7]:
service.wait_for_deployment(show_output=True)

Running....................
SucceededACI service creation operation finished, operation "Succeeded"


In [8]:
print('Service URI: {}'.format(service.scoring_uri))

Service URI: http://68559aa7-622b-48ab-ad29-bf9d6cd5c0f2.eastus.azurecontainer.io/score
